In [53]:
import torch
from models.experimental import attempt_load

model = attempt_load('yolov7-tiny.pt')
model.eval()

input_tensor = torch.rand((1,640,640,3)).to('cuda:0') * 255

out = model(input_tensor)
print(len(out))
print(out[0].shape)
print(len(out[1]))
print(out[1][0].shape)
print(out[1][1].shape)

Fusing layers... 
2
torch.Size([1, 25200, 85])
3
torch.Size([1, 3, 80, 80, 85])
torch.Size([1, 3, 40, 40, 85])


In [42]:
import torch

model = torch.jit.load('yolov7-tiny.torchscript.ptl')
input_tensor = torch.rand((1,640,640,3)) * 255

out = model(input_tensor)
print(len(out))
print(out[0].shape)
print(out[1][0].shape)
print(out[1][1].shape)

2
torch.Size([1, 25200, 85])
torch.Size([1, 3, 80, 80, 85])
torch.Size([1, 3, 40, 40, 85])


In [46]:
import onnx
import onnxruntime
import torch

onnx_model_path = "yolov7-tiny.onnx"

input_tensor = torch.rand((1,640,640,3)) * 255

onnx_model = onnx.load(onnx_model_path)  # load onnx model
onnx.checker.check_model(onnx_model)

ort_session = onnxruntime.InferenceSession(onnx_model_path)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# ONNX 런타임에서 계산된 결과값
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(input_tensor)}
ort_outs = ort_session.run(None, ort_inputs)

ort_outs[0].shape

(100, 7)

In [51]:
import numpy as np
import tensorflow as tf

batch_size, channels, height, width = 1, 3, 640, 640
input_shape = (batch_size, width, height, channels)

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path='mobilenetv1.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.uint8)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# get_tensor() returns a copy of the tensor data
# use tensor() in order to get a pointer to the tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)
print(output_data.shape)

[[[   0.037579    0.037054     0.98618     0.94172]
  [    0.28989    0.018194     0.98729     0.77295]
  [    0.35285     0.17184     0.98716     0.98866]
  [   0.070287  -0.0053692     0.96836     0.59008]
  [   0.024004     0.59542     0.96977      1.0188]
  [  -0.010242    -0.01757     0.24973      1.0113]
  [    0.60981    0.026421     0.98409      0.9851]
  [   0.042023  0.00097595     0.99275     0.25687]
  [   0.013653     0.28989     0.67876     0.98729]
  [   0.088822     0.39921     0.93188     0.99465]]]
(1, 10, 4)


In [ ]:
import torch

